In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image
from tensorflow.keras.preprocessing import image
import cv2

In [24]:
dimage=cv2.imread('clg.jpg')

In [25]:
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

# Display an image
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))

In [26]:
cv2.imshow("flight",dimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [28]:
names=['mixed3','mixed5']
layers=[base_model.get_layer(name).output for name in names]
dream_model=tf.keras.Model(inputs=base_model.input,outputs=layers)

In [29]:
def calculate_loss(img,model):
    img_batch=tf.expand_dims(img,axis=0)
    layer_activations=model(img_batch)
    if len(layer_activations)==1:
        layer_activations=[layer_activations]
        
    losses=[]
    for activation in layer_activations:
        loss=tf.math.reduce_mean(activation)
        losses.append(loss)
        
    return tf.reduce_sum(losses)

In [30]:
class Deepdream(tf.Module):
    
    def __init__(self,model):
        self.model=model
        
    @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
       )
    
    def __call__(self,img,steps,step_size):
        
        print("Tracing the loss")
        
        loss=tf.constant(0.0)
        
        for n in tf.range(steps):
            
            with tf.GradientTape() as tape:
                
                tape.watch(img)
                loss=calculate_loss(img,self.model)
                
            gradients=tape.gradient(loss,img)
            
            gradients /= tf.math.reduce_std(gradients) + 1e-8
            
            img=img+gradients*step_size
            
            img=tf.clip_by_value(img,-1,1)
            
        return loss,img

In [31]:
deepdream=Deepdream(dream_model)

In [32]:
def run_deepdream(img,steps=100,step_size=0.01):
    
    img=tf.keras.applications.inception_v3.preprocess_input(img)
    img=tf.convert_to_tensor(img)
    step_size = tf.convert_to_tensor(step_size)
    steps_remaining = steps
    step = 0
    while steps_remaining:
        if steps_remaining>100:
            run_steps = tf.constant(100)
        else:
            run_steps = tf.constant(steps_remaining)
        steps_remaining -= run_steps
        step += run_steps
        
        loss,img=deepdream(img,run_steps,tf.constant(step_size))
        
        #display.clear_output(wait=True)
        
        show(deprocess(img))
        
        print("Step {}, loss {}".format(step, loss))
        
    result = deprocess(img)
    #display.clear_output(wait=True)
    show(result)

    return result
        
    

In [ ]:
dreamimage=run_deepdream(img=dimage,steps=500,step_size=0.01)